This is a standalone notebook to solve a jigsaw puzzle

# Import dependencies

In [ ]:
# The jigsaw source image is available on reddit
# https://www.reddit.com/r/StuffMadeHere/comments/wi9nsn/inspired_by_shane_i_made_this_jigzilla_software/
!wget "https://preview.redd.it/gxijqxesq8g91.png?width=1135&format=png&auto=webp&v=enabled&s=c94d6483352aa00ef918b101ed56534683440fdd" -Ojigsaw144.png

In [ ]:
%pip install matplotlib opencv-python scipy ipyplot;

In [ ]:
import matplotlib.pyplot as plt
import ipyplot
import numpy as np
import cv2
import math
from functools import cache
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from collections import Counter

# Add utilities

In [ ]:
def imshow(img):
    plt.imshow(img[0:1680, 0:1700])

class Item():
    def __init__(self, **kwargs):
        self.update(**kwargs)

    def update(self, **kwargs):
        self.__dict__.update(kwargs)

class LoopingList(list):
    def __getitem__(self, i):
        if isinstance(i, int):
            return super().__getitem__(i % len(self))
        else:
            return super().__getitem__(i)

def plot_contour(contour, **kwargs):
    plt.plot(contour[:, :, 0], contour[:, :, 1], **kwargs)

def sub_contour(c, idx0, idx1):
    if idx1 > idx0:
        return c[idx0:idx1]
    else:
        return np.concatenate([c[idx0:], c[:idx1]])

# Detect contours

In [ ]:
img_rgb = cv2.imread("scans/jigsaw144.png")
h, w = img_rgb.shape[:2]
img_rgb = cv2.resize(img_rgb, (4*w, 4*h))
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
_, img_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
contours = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]
print("Number of detected contours: ", len(contours))

# Analyze pieces

In [ ]:
NB_SAMPLES = 20

pieces = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    # only keep pieces
    if cv2.contourArea(contour) < 10000:
        continue
    # compute piece name from its position
    col = int((x - w/2) * 13 / 4540)
    row = int(1 + (y - h/2) * 13 / 4450)
    name = chr(ord('A') + col) + str(row)
    # rotate the contour to vertical
    (cx, cy), (sx, sy), angle_degrees = cv2.minAreaRect(contour)
    cx = int(cx)
    cy = int(cy)
    sx = int(sx+20)
    sy = int(sy+20)
    if sy < sx:
        angle_degrees = (angle_degrees + 90) % 360
        sx, sy = sy, sx
    contour = contour.astype(np.float64)
    matrix = cv2.getRotationMatrix2D((cx, cy), angle_degrees, 1)
    contour = cv2.transform(contour, matrix)
    # translate the contour to have its center at (0, 0)
    contour = contour - np.array([cx, cy])
    # ensure corners are not at start or end of contour array
    distances = np.sum(contour**2, axis=2)[:, 0]
    min_idx = np.argmin(distances)
    contour = np.concatenate([contour[min_idx:], contour[:min_idx]])
    distances = np.concatenate([distances[min_idx:], distances[:min_idx]])
    # find peak distances
    peak_indices = find_peaks(distances, prominence=2000)[0]
    # find corners in peaks
    peaks = {}  # key=quarter, value=(diff_angle, idx)
    for idx in peak_indices:
        x, y = contour[idx, 0]
        radians = math.atan2(y, x)
        degrees = math.degrees(radians) % 360
        quarter = degrees // 90
        diff_degrees = degrees % 180
        if diff_degrees > 90:
            diff_degrees -= 180
        peaks.setdefault(quarter, []).append((abs(diff_degrees), idx))
    # get corner indices
    corner_indices = LoopingList()
    for quarter in range(4):
        idx = min(peaks[quarter])[1]
        corner_indices.append(idx)
    # compute edges
    edges = LoopingList()
    for quarter in range(4):
        idx0 = corner_indices[quarter + 1]
        idx1 = corner_indices[quarter]
        p0 = contour[idx0][0]
        p1 = contour[idx1][0]
        # normalize the contour: first point at (0, 0), last point at (X, 0)
        dx, dy = p1 - p0
        angle_radians = math.atan2(dy, dx)
        matrix = cv2.getRotationMatrix2D(p0, math.degrees(angle_radians), 1)
        normalized_piece_contour = cv2.transform(contour, matrix) - p0
        normalized_edge_contour = sub_contour(normalized_piece_contour, idx0, idx1 + 1)
        # compute the sign of the edge
        heights = normalized_edge_contour[:, 0, 1]
        if np.max(np.abs(heights)) > 10:
            sign = 1 if np.max(heights) > - np.min(heights) else -1
        else:
            sign = 0
        # compute the distance from the first point, this is not exactly edge.arc_length
        deltas = normalized_edge_contour[1:] - normalized_edge_contour[:-1]
        distances = np.cumsum(np.sqrt(np.sum(deltas**2, axis=2)))
        distance = distances[-1] / (NB_SAMPLES - 1)  # distance between 2 sample points
        # get N equidistant points
        sample_indices = (np.array([np.argmax(distances >= i*distance - 0.0001) for i in range(NB_SAMPLES)]) + idx0) % len(contour)

        edge = Item(
            idx0=idx0,
            idx1=idx1,
            sample_indices=sample_indices,
            normalized_piece_contour=normalized_piece_contour,
            sign=sign,
        )
        edges.append(edge)

    for idx, edge in enumerate(edges):
        edge.update(
            prev=edges[idx-1],
            next=edges[idx+1]
        )

    piece = Item(
        contour=contour,
        size=np.array([sy, sx]),
        edges=edges,
        name=name,
        nb_flats=len([edge for edge in edges if edge.sign == 0])
    )
    pieces.append(piece)
    
piece_by_name = dict([(piece.name, piece) for piece in pieces])

In [ ]:
import random
piece = random.sample(pieces, 1)[0]
# piece = piece_by_name['A12']
edge = random.sample(piece.edges, 1)[0]
print(piece.name)
plt.axis('equal')
plot_contour(piece.contour)
plot_contour(edge.normalized_piece_contour)
plot_contour(edge.normalized_piece_contour[edge.sample_indices], marker='x', ls='')
plt.axvline(x=0, c="gray", ls=":")
plt.axhline(y=0, c="gray", ls=":")
plt.show()

# Compute puzzle size

In [ ]:
def compute_size(area, perimeter):
    # perimeter = 2 * (H+W)
    # area = H*W
    # H**2 - perimeter/2 * H + area = 0
    a = 1
    b = -perimeter/2
    c = area
    delta = b**2 - 4*a*c
    h = int((-b - math.sqrt(delta)) / (2*a))
    w = int((-b + math.sqrt(delta)) / (2*a))
    return (min(h, w), max(h, w))

solution = Item()

nb_flats = Counter([piece.nb_flats for piece in pieces])
print(nb_flats)
assert nb_flats[2] == 4
area = len(pieces)
perimeter = nb_flats[1] + 2*nb_flats[2]
w, h = compute_size(area, perimeter)
print(f"Size of puzzle grid: {w} x {h}")
assert w * h == area
assert 2 * (w + h) == perimeter

solution.update(grid_size = (w, h))

# Compute the border

In [ ]:
after_flat_features = {}  # key=piece, value=features
before_flat_features = {}

for piece in pieces:
    for edge in piece.edges:
        if edge.sign == 0 and edge.prev.sign != 0:
            before_flat_features[piece] = edge.normalized_piece_contour[edge.prev.sample_indices][::-1]

        if edge.sign == 0 and edge.next.sign != 0:
            after_flat_features[piece] = edge.normalized_piece_contour[edge.next.sample_indices]

def piece_after_flat(piece0):
    features0 = after_flat_features[piece0]
    results = []
    for piece1, features1 in before_flat_features.items():
        diff = features1 - features0
        offset = np.mean(diff, axis=0)
        score = np.sum((diff - offset)**2)
        results.append((score, piece1))
    return min(results)[1]

first_piece = [piece for piece in pieces if piece.nb_flats == 2][0] 
ordered_border = [first_piece]
used_pieces = set()  # do not put first piece, it will be used for the last piece
while True:
    next_piece = piece_after_flat(ordered_border[-1])
    if next_piece in used_pieces:
        break
    ordered_border.append(next_piece)
    used_pieces.add(next_piece)

print("Computed border pieces:", ' '.join([piece.name for piece in ordered_border]))
assert len(used_pieces) == nb_flats[1] + nb_flats[2]
assert ordered_border[-1] == ordered_border[0]  # loop on the first piece
ordered_border = ordered_border[:-1]  # remove the repeated first piece
h, w = solution.grid_size
if ordered_border[h-1].nb_flats == 1:
    h, w = w, h
    solution.grid_size = w, h
assert [idx for idx, piece in enumerate(ordered_border) if piece.nb_flats == 2] == [0, h-1, h+w-2, 2*h+w-3]

# Place the border

In [ ]:
PAD = 30

solution.update(
    grid={} # key=(i, j), value=piece
)

def last_flat_edge(piece):
    return [edge for edge in piece.edges if edge.sign == 0 and edge.next.sign != 0][0]

def first_flat_edge(piece):
    return [edge for edge in piece.edges if edge.sign == 0 and edge.prev.sign != 0][0]

def place_piece(ij, piece, xy, idx, top_edge_idx, flat_edge, degrees):
    i, j = ij
    solution.grid[(i, j)] = piece
    dx, dy = piece.contour[flat_edge.idx1][0] - piece.contour[flat_edge.idx0][0]
    matrix = cv2.getRotationMatrix2D(piece.contour[idx][0], degrees + math.degrees(math.atan2(dy, dx)), 1)
    piece.contour = cv2.transform(piece.contour, matrix) + xy - piece.contour[idx][0]
    piece.edges = piece.edges[top_edge_idx:] + piece.edges[:top_edge_idx]

def place_border():
    pos = np.array([0, 0])
    xy = (0, 0)
    it_pieces = iter(ordered_border)
    for quarter, dpos in zip([0, 3, 2, 1, 0], [(0, 0), (1, 0), (0, 1), (-1, 0), (0, -1)]): 
        for piece in it_pieces:
            pos += dpos
            flat_edge = first_flat_edge(piece)  # this edge is aligned on previous piece
            top_edge_idx = piece.edges.index(flat_edge) + quarter
            if np.all(pos == 0):
                idx = flat_edge.idx0  # reference (xy) point is the corner
            else:
                idx = flat_edge.idx1
            place_piece(tuple(pos), piece, xy, idx, top_edge_idx, flat_edge, 90 * quarter)
            xy = piece.contour[last_flat_edge(piece).idx0]
            if piece.nb_flats == 2:
                break

place_border()

for piece in solution.grid.values():
   plt.axis('equal')
   plot_contour(piece.contour)

# Add inner pieces

In [ ]:
bottom_left_features = {}  # key=(piece, edge_idx), value=features

for piece in pieces:
    for idx, edge in enumerate(piece.edges):
        contour = edge.normalized_piece_contour
        sample_indices = np.concatenate([edge.next.sample_indices[::-1], edge.prev.prev.sample_indices[::-1]])
        bottom_left_features[(piece, idx)] = contour[sample_indices]
        # plot_contour(edge.normalized_piece_contour)
        # plot_contour(contour[sample_indices][:5], ls='', marker='o')

def edge_degrees(piece, edge_idx):
    contour = piece.contour
    edge = piece.edges[edge_idx]
    dx, dy = contour[edge.idx1][0] - contour[edge.idx0][0]
    return math.degrees(math.atan2(dy, dx))

def piece_bottom_left(piece_left, piece_bottom):
    points_left = piece_left.contour[piece_left.edges[3].sample_indices]
    # plot_contour(piece_left.contour)
    # plot_contour(points_left, ls='', marker='o')

    points_bottom = piece_bottom.contour[piece_bottom.edges[0].sample_indices]
    # plot_contour(piece_bottom.contour)
    # plot_contour(points_bottom, ls='', marker='o')

    angle_degrees = edge_degrees(piece_bottom, 0)
    matrix = cv2.getRotationMatrix2D(points_left[-1][0], angle_degrees, 1)
    features0 = cv2.transform(np.concatenate([points_left, points_bottom]), matrix)
    plot_contour(features0, ls='', marker='o')

    results = []
    for (piece1, edge1_idx), features1 in bottom_left_features.items():
        diff = features1 - features0
        offset = np.mean(diff, axis=0)
        score = np.sum((diff - offset)**2)
        results.append((score, piece1, edge1_idx))
        if score < 1000:
            plot_contour(features1 - offset)
    results.sort()
    print([r[0] for r in results[:5]])
    return min(results)[1:]

def place_at(i, j):
    piece_left = solution.grid[(i-1, j)]
    piece_bottom = solution.grid[(i, j+1)]
    piece, top_edge_idx = piece_bottom_left(solution.grid[(i-1, j)], solution.grid[(i, j+1)])
    bottom_edge = piece.edges[top_edge_idx+2]
    ref_point = piece_bottom.contour[piece_bottom.edges[0].idx0][0]
    place_piece((i, j), piece, ref_point, bottom_edge.idx1, top_edge_idx, bottom_edge, 180 - edge_degrees(piece_bottom, 0))

    # plot_contour(piece_left.contour)
    # plot_contour(piece_left.contour[piece_left.edges[3].sample_indices], ls='', marker='x')
    # plot_contour(piece_bottom.contour)
    # plot_contour(piece_bottom.contour[piece_bottom.edges[0].sample_indices], ls='', marker='x')
    # plot_contour(piece.contour)

# piece_at(1, 10)
# for i in range(2):
#     place_at(i+1, 10)
place_at(2, 10)

#for piece in solution.grid.values():
#   plot_contour(piece.contour)

#plot_contour(pieces[5].edges[2].normalized_piece_contour)
#plot_contour(pieces[5].edges[2].normalized_piece_contour[pieces[5].edges[2].sample_indices], ls='', marker='o')
# plot_contour(solution.grid[(i, j)].contour)
# plot_contour(solution.grid[(i, j)].contour)
# plot_contour(solution.grid[(i, j)].contour)
# plot_contour(solution.grid[(i, j)].contour)